In [11]:
using Knet: Knet, dir, accuracy, progress, sgd, load, save, gc, Param, KnetArray, Data, minibatch, nll, relu, training, dropout
using CUDA
using LinearAlgebra
using NBInclude
using CUDA

In [81]:
@nbinclude("deal_features.ipynb")
@nbinclude("model.ipynb")
@nbinclude("explore.ipynb")

explore (generic function with 1 method)

In [12]:
if CUDA.functional()
    atype = KnetArray{Float32};
else
    atype = Array{Float32};
end

KnetArray{Float32,N} where N

In [51]:
function opening_analysis(model,test_set)
    bid_dict=Dict()
    for i in 1:length(test_set)
        north=test_set[i][1]
        action,_=explore(model[1],atype(north),1,0)
        bid=bids[action]
        try
            push!(bid_dict[bid],north)
        catch
            bid_dict[bid]=[north]
        end
    end 
    return bid_dict
end

opening_analysis (generic function with 1 method)

In [8]:

deal_data=load("deals.jld2","deal_data_all");


In [94]:
model=load("model_median_feature.jld2","q_networks");

In [95]:
d=opening_analysis(model,deal_data)

Dict{Any,Any} with 14 entries:
  "2C" => [[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0,…
  "3C" => [[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 1.0, 0.0,…
  "2S" => [[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0  …  0.0, 0.0, 0.0,…
  "2N" => [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0,…
  "3H" => [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0,…
  "1H" => [[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 1.0,…
  "1D" => [[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0  …  0.0, 0.0, 0.0,…
  "2H" => [[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 1.0,…
  "1S" => [[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0,…
  "3S" => [[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0  …  0.0, 0.0, 0.0,…
  "1C" => [[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0,…
  "2D" => [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0  …  0.0, 0.0, 0.0

In [96]:
for bid in keys(d)
    hcps=[]
    n=length(d[bid]);
    dists=zeros(n,4);
    balanced_count=0

    for i in 1:n
        hand=d[bid][i]
        push!(hcps,hcp_of(hand))
        dists[i,:]=distribution_of(hand)
        balanced_count = balanced_count + is_balanced(hand)
    end
    println(bid,": ", minimum(hcps), " - ",maximum(hcps) ,", ", sum(dists,dims=1)./n, ", ", balanced_count/n)
    
end

2C: 14.0 - 27.0, [3.4035597529967307 2.0010897203051217 2.971304031965129 4.624046494733019], 0.4743915728296404
3C: 24.0 - 28.0, [2.2857142857142856 2.7142857142857144 4.357142857142857 3.642857142857143], 0.5714285714285714
2S: 17.0 - 28.0, [4.989637305699482 1.567357512953368 3.7797927461139897 2.6632124352331608], 0.24870466321243523
2N: 25.0 - 28.0, [2.25 3.0 4.125 3.625], 0.75
3H: 26.0 - 30.0, [3.75 4.5 3.0 1.75], 0.5
1H: 8.0 - 23.0, [3.7192039800995027 4.305737976782753 1.9507794361525705 3.024278606965174], 0.38925373134328356
1D: 8.0 - 20.0, [4.687718947052347 2.488739865879291 3.843559203282955 1.979981983785407], 0.34390951856671004
2H: 16.0 - 28.0, [4.192675159235669 4.101114649681529 2.070063694267516 2.636146496815287], 0.4434713375796178
1S: 10.0 - 23.0, [4.851942406954632 2.2242597120347734 3.4213528932355337 2.502444987775061], 0.4098071176310785
3S: 27.0 - 30.0, [5.0 3.0 2.0 3.0], 0.5
1C: 10.0 - 20.0, [2.602931709008072 3.0194374288119645 3.8694349527063836 3.50819590